In [138]:
import os
import pandas as pd

In [150]:
image_names_raw = open('dar_all_jpg_filenames.txt').read().split('\n')
image_names = set()

for name in image_names_raw:
    name = name.lower()
    if name.endswith('.jpg'):
        image_names.add(name.split('/')[-1])

In [35]:
csvs = []

for directory in os.walk('DAR/'):
    dirpath, _, files = directory
    for f in files:
        try:
            f = f.strip()
            if f.endswith('.csv') and 'MS-DAR-' in f and not f.startswith('.'):
                csvs.append(os.path.join(dirpath, f))
        except Error as e:
            print(e, f)
            break

In [47]:
good_csvs = []
for csv in csvs:
    
    with open(csv, encoding='latin8') as f:
        header = f.readline()
        if len(header) > 500:
            print(f + " is fucked")

        if 'filename' in header.lower():
            good_csvs.append(csv)

In [52]:
dfs = []
for csv in good_csvs:
    dfs.append(pd.read_csv(csv, encoding='latin8'))

In [54]:
data = pd.concat(dfs)

In [72]:
for col in data.columns:
    if data[col].isnull().mean() > 0.95:
        del data[col]

In [77]:
data_short = data[['eid', 'filename', 'class-mark', 'sort-class-mark']]

In [80]:
data_short.drop_duplicates(inplace=True)

/usr/local/lib/python3.5/dist-packages/pandas/util/decorators.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)


In [88]:
for col in data_short.columns:
    print(col, data_short[col].nunique(), data_short[col].isnull().sum())
    
data_short = data_short[data_short.filename.notnull()]
data_short.to_csv('file_identifiers.csv', index=False)

eid 37161 0
filename 37710 0
class-mark 37044 0
sort-class-mark 37082 560


In [117]:
data_short.groupby('filename').count().eid.value_counts()

1    36214
2      998
3      440
4       52
5        6
Name: eid, dtype: int64

In [131]:
final_data = []

for name, group in data_short.groupby('filename'):
    if len(group) == 1:
        final_data.append(group.values.tolist()[0])
    
    eid = set(group.eid.dropna().values)
    class_mark = set(group['class-mark'].dropna().values)
    sort_class_mark = set(group['sort-class-mark'].dropna().values)

    if len(eid) > 1 or len(class_mark) > 1 or len(sort_class_mark) > 1:
        
        continue

In [135]:
with open('file_identifiers.csv', 'w') as f:
    f.write('\t'.join(['eid', 'filename', 'class_mark', 'sort_class_mark']) + '\n')
    
    for row in final_data:
        
        f.write('\t'.join([str(int(row[0]))] + row[1:]) + '\n')

In [136]:
!head file_identifiers.csv

eid	filename	class_mark	sort_class_mark
128555	MS-DAR-00002-000-00001.jpg	DAR 2: fc	DAR 00002: 00000fc
128556	MS-DAR-00002-000-00002.jpg	DAR 2: ifc	DAR 00002: 00000ifc
108865	MS-DAR-00002-000-00197 1.jpg	DAR 2: 75[3]_a	DAR 00002: 00075 00003A
108869	MS-DAR-00002-000-00205 1.jpg	DAR 2: 77[2]_a	DAR 00002: 00077 00002A
128557	MS-DAR-00002-000-00563.jpg	DAR 2: ibc	DAR 00002: 00999ibc
128558	MS-DAR-00002-000-00564.jpg	DAR 2: bc	DAR 00002: 00999z_bc
128559	MS-DAR-00002-000-00565.jpg	DAR 2: sp	DAR 00002: 00999z_sp
128273	MS-DAR-00003-000-00001.jpg	DAR 3: fc	DAR 00003: 00000fc
128274	MS-DAR-00003-000-00002.jpg	DAR 3: ifc	DAR 00003: 00000ifc


Relates the image "filename" to the "eid" (useful for finding the images at their URLS), "class_mark" (useful for relating the images to the text transcript) and to "sort_class_mark" (which might be useful!). NOTE: Strip your filenames before searching! Some filenames have leading spaces (yes, actually) as the spaces are not included here.

# Let's find some reverse things

In [141]:
data = pd.read_csv('file_identifiers.csv', encoding='latin8', sep='\t')

In [147]:
data.sort_class_mark.str.lower().str.endswith('v').mean()

0.47525818744132103

In [148]:
data.sort_class_mark.str.lower().str.endswith('r').mean()

0.48125034517037607

In [149]:
from collections import Counter
Counter(v[-1] for v in data.sort_class_mark.str.lower().values)

Counter({"'": 4,
         '0': 62,
         '1': 115,
         '2': 114,
         '3': 68,
         '4': 62,
         '5': 54,
         '6': 59,
         '7': 64,
         '8': 68,
         '9': 64,
         'a': 64,
         'b': 66,
         'c': 426,
         'd': 47,
         'e': 43,
         'f': 45,
         'g': 6,
         'h': 6,
         'i': 2,
         'j': 2,
         'k': 1,
         'l': 12,
         'n': 4,
         'p': 90,
         'r': 17428,
         't': 11,
         'u': 1,
         'v': 17211,
         'x': 15})

In [162]:
len(set(data.filename.str.lower().values).intersection(image_names))#/len(image_names)

29633